# 5. Grundlagen der Optimierung und Gradient Descent
<a href="https://de.wikipedia.org/wiki/Optimierung_(Mathematik)" target="_blank">Optimierung</a> begegnet uns nahezu überall in modernen Problemen und interessanten Fragestellungen. Wir werden uns in diesem Notebook mit den Grundlagen der Optimierung befassen und dann auch noch einen konkreten Optimierungs-Algorithmus kennen lernen: <a href="https://de.wikipedia.org/wiki/Gradientenverfahren" target="_blank">Gradient Descent</a>.

## 5.1 Was ist Optimierung
Bei einem Optimierungsproblem geht es grundsätzlich darum, eine bestimmte Größe oder Funktion von Input-Variablen entweder zu maximieren oder zu minimieren. Diese Größe heißt je nach Kontext z.B. "Zielfunktion", "Kostenfunktion", "Fitnessfunktion", etc. Gemeint ist damit aber einfach immer jene Funktion, die optimiert werden soll.

Grundsätzlich kann es bei Optimierungsproblemen verschiedene Einschränkungen geben:

* Die erlaubten Wertebereiche der Input-Variablen können eingeschränkt sein
* Die Wertebereiche der Zielfunktion können eingeschränkt sein
* Die Werte der Input-Variablen können durch eine sogenannte Nebenbedingung zusammenhängen
* Der "Anspruch" der Optimierung kann eingeschränkt sein auf entweder _lokale_ oder _globale_ Optima, die man finden möchte.


Sehen wir uns gleich ein __einfaches Beispiel__ an, dass zwei dieser Einschränkungen aufweist:

Finde jene beiden verschiedenen <a href="https://de.wikipedia.org/wiki/Zahl" target="_blank">Zahlen</a> aus der Menge der natürlichen Zahlen bis 10, die folgendes leisten:

* Die Summe der beiden Zahlen muss 10 ergeben
* Das Produkt der beiden Zahlen soll maximal sein


Die eine Einschränkung ist die <a href="https://de.wikipedia.org/wiki/Nebenbedingung" target="_blank">Nebenbedingung</a> der fixen Summe, die andere Einschränkung ist, dass wir es mit natürlichen Zahlen zu tun haben, also einer diskreten (bzw. letztlich auch endlichen) Menge von möglichen Input-Werten.

Um dieses Problem zu lösen müssten wir eigentlich gar nicht programmieren, sondern nur nachdenken. Aber ich werde trotzdem das Notebook nutzen, um das Beispiel zu illustrieren.

In [ ]:
%matplotlib notebook

# zunächst die Imports für diese Einheit
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import sympy as sp


In [ ]:
# gib alle natürlichen Zahlen von 1 bis 10 aus
number_array = np.arange(1,11,1)
print("Kanidaten:", number_array)

# leere Liste, wo alle Paare hineinsollen, die die Bedingungen erfüllen
pair_list = []

# nun gehen wir alle Kombinationen durch
for number_1 in number_array:
    
    for number_2 in number_array:
        
        # überprüfe die Summenvoraussetzung und die Ungleichheit
        if (number_1 + number_2 == 10) and (number_1 != number_2):
            # das ist interessant, gib die Zahlen und das Produkt aus
            print(number_1, "mal", number_2, "ist", number_1 * number_2)
            
            # hänge diese Outputs auch an die Liste an
            pair_list.append([number_1, number_2, number_1 * number_2])
        
# verwandle die Liste in ein Array
pair_array = np.array(pair_list)

# zum Schluss, finde das Maximum in der Liste
optimal_position = np.argmax(pair_array[:, -1])

optimal_solution = pair_array[optimal_position]

print("Die optimale Lösung:", optimal_solution)

## 5.2 Varianten der Optimierung
Hier haben wir also gesehen, wie man, mehr oder weniger mit der Hand und eine nach der anderen, alle Möglichkeiten durchspielt und am Schluss nachsieht, welche Möglichkeit optimal ist. So eine herangehensweise nennt man oft <a href="https://de.wikipedia.org/wiki/Brute-Force-Methode" target="_blank">"brute-force"</a> approach, denn mehr als Rechengewalt haben wir nicht verwendet. Insbesondere haben wir folgendes __nicht__ getan:

* Eine Formel verwendet, um die Lösung direkt zu bestimmen
* Einen iterativen <a href="https://de.wikipedia.org/wiki/Algorithmus" target="_blank">Algorithmus</a> verwendet, der von einem Startwert aus Schritt für Schritt immer bessere Lösungsvorschläge liefert
* Die Werte der Input-Variablen eingeschränkt, als wir sie in die Loops geschickt haben
* Aus möglichen Input-Werten per Zufallsprinzip <a href="https://theoreticalphysics.eu/monte-carlo-methoden-simulation-und-integration/" target="_blank">gesampelt</a>, um Rechenzeit zu sparen, aber gleichzeitig trotzdem einen repräsentativen Teil aller Möglichkeiten abzubilden


Diese Dinge sind natürlich grundsätzlich gute Möglichkeiten, um ein Optimierungsproblem zu vereinfachen. Machmal sind sie allerdings sogar unerlässlich, um überhaupt Fortschritte erzielen zu können. Nehmen wir dazu nocheinmal das obige Beispiel zur Hand und stellen wir uns vor, aus 10 würde eine viel größere Zahl.

Aus unserer <a href="https://de.wikipedia.org/wiki/Zeitkomplexit%C3%A4t" target="_blank">Zeit-Komplexitäts</a>-Analyse wissen wir noch, dass ein Programm mit zwei Loops bis $N$ zunächst einmal wie $N^2$ skalieren wird. Hier kommt dann noch etwas dazu, nämlich die Suche nach dem Maximum in der entstandenen Liste. Uns geht es hier allerdings nicht so sehr um diese Details, sondern zunächst einmal darum, dass wir (auch von den Primzahlen damals) schon wissen, dass man hier im Prinzip ganz ordentlich einsparen kann, wenn man z.B. die Loops gut einschränken kann.

Wenn wir hier z.B. den inneren Loop nur bis zu 1 unter der aktuellen Zahl im äußeren Loop laufen lassen, dann können wir sogar die Hälfte der if-Abfrage weglassen (nämlich ob die beiden Zahlen gleich sind). Somit ändern wir den Code etwas ab zu:

In [ ]:
# wo liegt die Grenze?
n = 1000

number_array = np.arange(1, n+1, 1)
# print("Kanidaten:", number_array)

# leere Liste, wo alle Paare hineinsollen, die die Bedingungen erfüllen
pair_list = []

# nun gehen wir nur mehr die Hälfte aller Kombinationen durch
for number_1 in number_array:
    
    # der Zweite Loop wird jetzt früher abgebrochen
    for number_2 in number_array[:number_1-1]:
        
        # überprüfe die Summenvoraussetzung
        if (number_1 + number_2 == n):
            # das ist interessant, gib die Zahlen und das Produkt aus
            # print(number_1, "mal", number_2, "ist", number_1 * number_2)
            
            # hänge diese Outputs auch an die Liste an
            pair_list.append([number_1, number_2, number_1 * number_2])

# verwandle die Liste in ein Array
pair_array = np.array(pair_list)

# zum Schluss, finde das Maximum in der Liste
optimal_position = np.argmax(pair_array[:, -1])

optimal_solution = pair_array[optimal_position]

print("Die optimale Lösung:", optimal_solution)

## 5.3 Optimierung mit kontinuierlichen Variablen
Unsere ersten Optimierungserfahrungen in diesem Notebook haben wir also mit diskreten Variablen gemacht. Oft werden Sie allerdings mit <a href="https://de.wikipedia.org/wiki/Kontinuum_(Mathematik)" target="_blank">kontinuierlichen</a> Variablen (z.B. Koordinaten) zu tun haben. Daher wenden wir uns nun dieser Situation zu und befassen uns gleich auch noch mit einem konkreten iterativen Lösungsalgorithmus dafür.

Zunächst sehen wir uns einmal so ein Problem konkret an __einem Beispiel__ an. Nehmen wir z.B. folgendes: 

Gegeben ist die Funktion $f(x, y) = (x - 1)^2 + (y-2)^2 + 1$. Für welche $(x, y)$ ist $f$ minimal?

Dazu noch ein paar Anmerkungen: 

* Üblicherweise kann man für Optimierungsprobleme mit kontinuierlichen Variablen die Wertebereiche der Input-Variablen einfach als $\mathbb{R}$ annehmen (oder $\mathbb{R}$, eingeschränkt auf ein Intervall). 
* Man muss aufpassen, dass man zwischen _lokalen_ und _globalen_ Optima unterscheidet. Ein lokales Minimum kann man z.B. mit Methoden aus der <a href="https://de.wikipedia.org/wiki/Differentialrechnung" target="_blank">Differenzialrechung</a> finden. Das muss jedoch nicht unbedingt auch das globale Minimum sein.
* Insbesondere bei der Einschränkung der Input-Werte auf Intervalle muss man von vornherein wissen, ob man (ggf. eher) nach globalen oder lokalen Optima sucht.


In unserem Beispiel suchen wir nach beidem gleichzeitig, denn die beiden sind identisch. Das können wir allerdings nur sagen, weil wir wissen, wie die Funktion in etwa aussieht. Hier ist sie z.B. einmal geplottet:


In [ ]:
# definiere f
def our_function(x, y):
    # Beachte: Die Argumente können hier Arrays sein
    return (x-1)**2 + (y-2)**2 + 1

# definiere x-Werte
x_vals = np.linspace(-1, 3, 50)

# definiere y-Werte
y_vals = np.linspace(0, 4, 50)

# erzeuge x-y-Grid für 3D Plots
X, Y = np.meshgrid(x_vals, y_vals)

Z = our_function(X, Y)

# neue Grafik
fig = plt.figure()

# 3D Achsen erzeugen
ax = fig.add_subplot(1,1,1, projection='3d')

# erzeuge 3D-Oberflächenplot
ax.plot_surface(X, Y, Z, cmap="magma")

plt.show()

So sieht also die Funktion aus. Es gibt auch noch andere Arten, so etwas zu plotten, z.B. diese:

In [ ]:
# neue Grafik
fig = plt.figure(figsize=(10, 5))

# 3D Achsen 1 erzeugen
ax1 = fig.add_subplot(1,2,1, projection='3d')

# erzeuge 3D-Oberflächenplot
ax1.scatter(X, Y, Z, s=0.1)

# 2D Achsen 2 erzeugen
ax2 = fig.add_subplot(1,2,2)

# erzeuge 2D-Contourplot
ax2.contour(X, Y, Z, 50, cmap='magma')

plt.show()

Wie kann man nun von so einer Funktion das Minimum finden? Beim <a href="https://theoreticalphysics.eu/vektoren-matrizen-und-vektorisierung-in-python/" target="_blank">Kapitel über Vektoren und Matrizen</a> hatten wir schon einmal so etwas ähnliches, nämlich über die <a href="https://de.wikipedia.org/wiki/Lineare_Regression" target="_blank">lineare Regression</a> (Zur besseren Erklärung: Dort war die Zielfunktion die Summe aller quadrierten Abstände der Beschreibung von den Datenpunkten). 

Wir haben damals die exakte Lösungsformel für den <a href="https://de.wikipedia.org/wiki/Methode_der_kleinsten_Quadrate" target="_blank">Least-Squares-Fit</a> aufgeschrieben und auch umgesetzt. Das ginge konkret auch bei dieser Form der Funktion (weil sie auch quadratisch ist), aber das geht nicht immer. Konkret muss man sogar sagen, dass sehr viele Optimierungsprobleme bisher keine Lösung in dem Sinn haben, dass kein Algorithmus bekannt ist, der das echte Optimum findet.

Nichtsdestotrotz sind viele dieser Probleme zumindest _näherungsweise_ lösbar, und das ist meist gut genug. Wir werden uns daher hier einem Algorithmus zuwenden, der allgemein einsetzbar ist, und der zumindest näherungsweise Lösungen finden kann.

## 5.4 Gradient Descent

Beim Gradient-Descent-Algorithmus geht es darum, bei einer Minimierungsaufgabe dem Gradienten der (Hyper-)Fläche der Zielfunktion im Raum der Variablen Schritt für Schritt so zu folgen, dass man "immer bergab" geht. Stellen Sie sich vor, Sie stehen an einer riesigen Grube (z.B. einem Meteoritenkrater) und wollen an den tiefsten Punkt kommen. Dann könnten Sie folgendes tun:

* Suchen Sie rund um Ihre Position herum jene Stelle, wo es am meisten bergab geht
* Gibt es überhaupt eine Richtung, in der es bergab geht?
  * Ja? Machen Sie einen Schritt in diese Richtung
  * Nein? Sie haben ein (lokales) Minimum erreicht


Mathematisch ausgedrückt macht man bei dieser Vorgehensweise folgendes:

* Wählen Sie einen Startpunkt $(x_0, y_0)$
* Wählen Sie eine Schrittweite $a$
* Berechnen Sie dort den Gradienten $\nabla f(x, y)|_{(x=x_0, y=y_0)}$
* Berechnen Sie den nächsten Punkt, einen Schritt vom vorigen Punkt entfert, entlang des negativen Gradienten, also
$$(x_1, y_1) = (x_0, y_0) - a \nabla f(x, y)|_{(x=x_0, y=y_0)}$$
* Nutzen Sie jetzt $(x_1, y_1)$ als neuen Startpunkt für den nächsten Schritt und wiederholen Sie das, bis die Abbruchbedingung erfüllt ist
* Abbruchbedingung: Eine vorgegebene maximale Anzahl von Schritten oder irgendwann ist $||(x_n, y_n)-(x_{n-1}, y_{n-1})||<\varepsilon$ für eine vorgegebene Genauigkeit $\varepsilon$.


Wir wollen nun diesen Algorithmus für unser obiges konkretes Beispiel durchgehen.


In [ ]:
# definiere zwei Variablen x und y über SymPy
x, y = sp.symbols('x, y')

# definiere die Funktion f
f = (x-1)**2 + (y-2)**2 + 1

# Startwert 
x0 = np.array([0.5, 1.])

# übergebe an wachsendes Array für den Pfad
xy = np.array([x0])

# Schrittweite
a = 0.1

# Loop über Schritte (wir lassen hier den Genauigkeitscheck einfach mal weg)
for ind in range(100):
    
    # hole den letzten Punkt aus der Punkteliste (Pfad)
    present_point = xy[-1]
    
    # schreibe den momentanen Punkt heraus
    print(present_point)

    # berechne den Gradienten and dieser Stelle
    gradi = np.array([sp.diff(f, x).subs(x, present_point[0]).subs(y, present_point[1]),
                      sp.diff(f, y).subs(x, present_point[0]).subs(y, present_point[1])])
    
    # mache einen Gradient-Descent Schritt, also ziehe den Gradienten mal Schrittweite
    # vom derzeitigen Punkt ab
    # hänge dann das Resultat an das xy-Array an
    xy = np.append(xy, np.reshape(present_point - a * gradi, (1, 2)), axis=0)
    
    

In [ ]:
# sehen wir uns das an
fig = plt.figure()

# 2D Achsen 2 erzeugen
ax = fig.add_subplot(1,1,1)

# erzeuge 3D-Oberflächenplot
ax.contour(X, Y, Z, 50, cmap='magma')

# erzeuge x und y Werte aus dem Pfad
x_vals, y_vals = np.transpose(xy)

# Plotte zusätzlich den Pfad
ax.plot(x_vals, y_vals, "rx-", markersize=10)

# Plot anzeigen
plt.show()

## 5.5 Übungsaufgabe: Spielen Sie mit dem Gradient-Descent-Algorithmus und dem Plot
Kopieren Sie nun einfach die relevanten Code-Schnipsel von oben und probieren Sie folgende Dinge aus:

* Nehmen Sie eine andere Funktion $f(x, y)$ und probieren Sie aus, was passiert
* Passen Sie auch die Wertebereiche für $x$ und $y$ entsprechend an, sodass Sie im Plot sehen, wo der Algorithmus hinläuft
* Experimentieren Sie auch mit der Schrittgröße:
  * Was passiert, wenn $a$ zu klein gewählt wird?
  * Was passiert, wenn $a$ zu groß gewählt wird?
* Finden Sie eine Funktion $f$, für die dieser Algorithmus nicht konvergiert?


In [ ]:
# definiere eine andere Funktion g
g = sp.sin((x-1)**2 + (y-2)**2)**2

# Startwert 
x0 = np.array([0.5, 0.7])

# übergebe an wachsendes Array für den Pfad
xy = np.array([x0])

# Schrittweite
a = 0.05

# Loop über Schritte (wir lassen hier den Genauigkeitscheck einfach mal weg)
for ind in range(20):
    
    # hole den letzten Punkt aus der Punkteliste (Pfad)
    present_point = xy[-1]
    
    # schreibe den momentanen Punkt heraus
    print(present_point)

    # berechne den Gradienten and dieser Stelle
    gradi = np.array([sp.diff(g, x).subs(x, present_point[0]).subs(y, present_point[1]),
                      sp.diff(g, y).subs(x, present_point[0]).subs(y, present_point[1])])
    
    # mache einen Gradient-Descent Schritt, also ziehe den Gradienten mal Schrittweite
    # vom derzeitigen Punkt ab
    # hänge dann das Resultat an das xy-Array an
    xy = np.append(xy, np.reshape(present_point - a * gradi, (1, 2)), axis=0)
    
    

In [ ]:
# passe Plotbereiche an

# definiere x-Werte
x_vals = np.linspace(0, 2, 250)

# definiere y-Werte
y_vals = np.linspace(0, 3, 250)

# erzeuge x-y-Grid für 3D Plots
X, Y = np.meshgrid(x_vals, y_vals)

# erzeuge Funktion für numerische Auswertung automatisch
# sodass sie für den Plot übereinstimmt
g_num = sp.lambdify([x, y], g)

# erzeuge Z-Werte über die lambdifizierte Funktion g
Z = g_num(X, Y)



# sehen wir uns auch das an
fig = plt.figure()

# 2D Achsen 2 erzeugen
ax = fig.add_subplot(1,1,1)

# erzeuge 3D-Oberflächenplot
ax.contour(X, Y, Z, 50, cmap='magma')

# erzeuge x und y Werte aus dem Pfad
x_vals, y_vals = np.transpose(xy)

# Plotte zusätzlich den Pfad
ax.plot(x_vals, y_vals, "rx-", markersize=10)

# Plot anzeigen
plt.show()